### Recommender System

&nbsp;

Recommender system is a system widely used in Amazon, Netflix to predict user rating for a given item. Usually the system involves collaborative filtering, content-based filtering, session-based filtering or even a mixture.

* Collaborative filtering seeks connections across different users and items to predict the rating. It is more related to unsupervised learning. Even inside collaborative filtering, there are a few different techniques such as model-based (matrix factorization, latent variables), memory-based (baseline, KNN) and even a hybrid of both. Collaborative filtering will be the main focus of this script. 
* Content-based filtering collects user and item profile. Based upon the features in the profile, it forecasts the user preference towards different items. It is more related to supervised learning.
* Session-based filtering monitors the user interaction within a session to create recommendations. It is quite helpful to navigate through the cold start problem where a new user has not much available information for modelling. It is more related to deep learning, in particular, Recurrent Neural Network.

In a recommender system, not every customer rates every item so it effectively forms a partially filled customer vs item matrix. To recommend anything to the existing customer, a fully filled matrix must be inferred from the dataset. This falls into the spectrum of matrix completion. The most popular sub-problem of matrix completion is to find a low rank matrix via convex optimization (Candès and Recht, 2008). The sub-problem assumes there must be latent variables influencing the users and the items in the matrix. Thus, the matrix must be low rank.

Reference to matrix completion

https://github.com/je-suis-tm/machine-learning/blob/master/matrix%20completion.ipynb

Reference to KNN

https://github.com/je-suis-tm/machine-learning/blob/master/k%20nearest%20neighbors.ipynb

In [1]:
import os
os.chdir('K:/ecole/github/televerser/matrix completion')
import pandas as pd
import numpy as np
import numba

### Read Files

In [2]:
#data comes from the link below
# http://files.grouplens.org/datasets/movielens/ml-100k/u.data
# https://github.com/je-suis-tm/machine-learning/blob/master/data/movielens.csv
df=pd.read_csv('movielens.csv')

In [3]:
#shrink data size for better performance
df=df[df['user']<=100][df['item']<=500].copy()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [4]:
#convert array into matrix form
matrix=df.pivot(index='item',columns='user',values='rating')
arr=np.array(matrix)

### Funk SVD

&nbsp;

Simon Funk developed a SVD-like latent factor model to win 3rd prize in 2006 Netflix problem. Conventionally people call it Funk SVD, although it is merely inspired by Singular Value Decomposition without explicitly using SVD. Funk SVD is really easy to implement and quick to converge with high accuracy. It is a type of collaborative filtering focusing on matrix factorization. The official optimization problem is formulated as below.

$$ \min_{p_*,q_*,b_*}\,\sum_{r_{ui}\,\in\,\mathcal{K}} \left(r_{ui} - \hat{r}_{ui} \right)^2 + \lambda( ||p_u||^2 + ||q_i||^2 + b_u^2 + b_i^2 )$$

where 

${r}_{ui}$ denotes the rating of item $i$ by user $u$

$\hat{r}_{ui}$ denotes the estimated rating of item $i$ by user $u$, it can be decomposed into the form of $\mu + b_u + b_i + p_u^Tq_i$

$\mathcal{K}$ denotes the observed user vs item matrix (partially filled)

$\mu$ denotes overall average rating

$b_u$ denotes the deviation from overall average rating caused by user $u$, can be seen as user baseline where some users are more critical so they give lower ratings in general

$b_i$ denotes the deviation from overall average rating caused by item $i$, can be seen as item baseline where some items are more appealing so they attract higher ratings in general

$p_u$ denotes the latent factors of user $u$, in translation, user preference

$q_i$ denotes the latent factors of item $i$, in translation, item attributes

$\lambda$ denotes LaGrange multiplier which is the coefficient of L2 penalty function

&nbsp;

To explain a bit, Funk SVD assumes that both user and item are affected by $m$ number of latent factors ($p_u \in \mathbb{R}^{m}\,\&\,q_i \in \mathbb{R}^{m}$). These latent factors could be percentage of action in the movie, number of tier 1 hollywood stars, etc. The rating $r_{ui}$ is merely the linear combination $p_u^Tq_i$ of user preference and item attributes. This decomposition is similar to SVD in the form of $U\Sigma V^T$ without the eigenvalue diagonal $\Sigma$ as scaling factors. Hence, the first bit of the objective function is an ordinary least square to minimize the sum of squared error between existing rating and estimated rating. Since the matrix is partially filled, the second bit of the objective function is L2 norm regularization on user preference $p_u$, item attributes $q_i$, user deviation $b_u$ and item deviation $b_i$ to avoid overfit problem.

The actual optimization solver is inspired by gradient descent $\theta:=\theta-\alpha\frac{\partial J(\theta)}{\partial \theta}$. Applying the same logic to every unknown parameters $p_u$, $q_i$, $b_u$ and $b_i$, we end up with the following updates.

$$b_u := b_u + \alpha (\epsilon_{ui} - \lambda b_u)$$
$$b_i := b_i + \alpha (\epsilon_{ui} - \lambda b_i)$$
$$p_u := p_u + \alpha (\epsilon_{ui} \cdot q_i - \lambda p_u)$$
$$q_i := q_i + \alpha (\epsilon_{ui} \cdot p_u - \lambda q_i)$$

where

$\epsilon_{ui}$ denotes the error between existing rating and estimated rating in the form of $r_{ui} - \hat{r}_{ui}$

$\alpha$ denotes the learning rate of gradient descent which dictates how soon the solver reaches the local optima

&nbsp;

Reference to Singular Value Decomposition

https://github.com/je-suis-tm/machine-learning/blob/master/principal%20component%20analysis.ipynb

Reference to Gradient Descent

https://github.com/je-suis-tm/machine-learning/blob/master/gradient%20descent.ipynb

Reference to Simon Funk's blog

https://sifter.org/~simon/journal/20061211.html

###### Functions

In [5]:
#use numba to dramatically boost the speed of linear algebra
#this will be a lot faster than surprise library
@numba.njit
def funk_svd_epoch(arr,miu,b_u,b_i,p_u,q_i,alpha,lambda_):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue

            #compute error
            epsilon_ui=r_ui-miu-b_u[u]-b_i[i]-p_u[u].T@q_i[i]
            error+=epsilon_ui

            #update
            b_u[u]+=alpha*(epsilon_ui-lambda_*b_u[u])
            b_i[i]+=alpha*(epsilon_ui-lambda_*b_i[i])
            p_u[u]+=alpha*(epsilon_ui*q_i[i]-lambda_*p_u[u])
            q_i[i]+=alpha*(epsilon_ui*p_u[u]-lambda_*q_i[i])
    
    return error,b_u,b_i,p_u,q_i

In [6]:
#svd inspired latent factor model by simon funk
def funk_svd(arr,miu_init=None,b_u_init=[],b_i_init=[],
             p_u_init=[],q_i_init=[],num_of_rank=40,
             alpha=0.005,lambda_=0.02,tau=0.0001,
             max_iter=20,diagnosis=True
             ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
        
    #user latent factors
    if len(p_u_init)==0:
        p_u=np.zeros((arr.shape[1],num_of_rank))
        p_u.fill(0.1)
    else:
        p_u=p_u_init
    
    #item latent factors
    if len(q_i_init)==0:
        q_i=np.zeros((arr.shape[0],num_of_rank))
        q_i.fill(0.1)
    else:
        q_i=q_i_init
    
    #gradient descent
    while not stop:
        
        error,b_u,b_i,p_u,q_i=funk_svd_epoch(arr,miu,
                                             b_u,b_i,
                                             p_u,q_i,
                                             alpha,lambda_)

        counter+=1

        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged. Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error**2
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i,p_u,q_i

###### Run

In [7]:
#initialize
num_of_latent_factors=40
max_num_of_epoch=500
learning_rate=0.01
lagrange_multiplier=0.02
tolerance=0.005

In [8]:
#funk svd
b_u,b_i,p_u,q_i=funk_svd(arr,num_of_rank=num_of_latent_factors,
         alpha=learning_rate,
         lambda_=lagrange_multiplier,tau=tolerance,
         max_iter=max_num_of_epoch)

36 iterations to reach convergence



In [9]:
#compute global mean
miu=arr[~np.isnan(arr)].mean()

#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),arr.shape[1],axis=1)+q_i@p_u.T

In [10]:
#use mse as benchmark for comparison
mse_funk_svd=np.square((
    output-arr)[~np.isnan(arr)]).sum()/len(arr[~np.isnan(arr)])
print('Funk SVD Mean Squared Error:',round(mse_funk_svd,3))

Funk SVD Mean Squared Error: 0.726


### SVD++

&nbsp;

Inspired by Funk SVD, more and more variations have been invented to enhance the prediction accuracy and the convergence speed. One of the most famous variation is SVD++ (Koren, 2008). It can be seen as an augmented Funk SVD with user implicit feedback. The accuracy is further improved at the cost of implicit feedback integration. The official optimization problem is formulated as below. The notations are almost the same with only a few extra from implicit feedback.

$$ \min_{p_*,q_*,b_*}\,\sum_{r_{ui}\,\in\,\mathcal{K}} \left(r_{ui} - \mu - b_u - b_i - q_i^T\left(p_u+|N(u)|^{-\frac{1}{2}} \sum_{j \in N(u)}y_j\right) \right)^2 + \lambda\left( ||p_u||^2 + ||q_i||^2 + \sum_{j \in N(u)}||y_j||^2 + b_u^2 + b_i^2 \right)$$

where 

$N(u)$ denotes the implicit preference list of user $u$

$y_j$ denotes the item $j$ that user $u$ implicitly prefers

&nbsp;

To explain a bit, SVD++ incorporates user implicit feedback into the model. Users may have watched a lot of movies on Netflix yet they choose to rate items in $N(u)$ for a particular reason. For instance, user $u$ may have watched six seasons of House of Cards but he strongly resents the last season. Hence, he gives one star rating to the last season. This kind of implicit feedback is not captured in any latent factors. Nonetheless, the implicit feedback $y_j$ is not independent of any latent factors. Since certain combination $q_i^Tp_u$ triggers the user response and amplifies the extreme rating, $y_j$ is included in both the objective function and the constraint to avoid overfit problem. FYI, $y_j$ can be seen as another latent factor vector with the exact dimension as $q_i$.

The actual optimization solver is similar to Funk SVD with one more parameter $y_j$. 

$$b_u := b_u + \alpha (\epsilon_{ui} - \lambda b_u)$$
$$b_i := b_i + \alpha (\epsilon_{ui} - \lambda b_i)$$
$$p_u := p_u + \alpha (\epsilon_{ui} \cdot q_i - \lambda p_u)$$
$$q_i := q_i + \alpha (\epsilon_{ui} \cdot (p_u+|N(u)|^{-\frac{1}{2}} \sum_{j \in N(u)}y_j) - \lambda q_i)$$
$$y_j := y_j + \alpha (|N(u)|^{-\frac{1}{2}} \cdot \epsilon_{ui} \cdot q_i - \lambda y_j)$$

&nbsp;

Reference to the original paper of SVD++ (equation 15)

https://people.engr.tamu.edu/huangrh/Spring16/papers_course/matrix_factorization.pdf

###### Functions

In [11]:
#use numba to dramatically boost the speed of linear algebra
#this will be a lot faster than surprise library
@numba.njit
def svd_plus_plus_epoch(arr,miu,b_u,b_i,p_u,q_i,y_j,alpha,lambda_):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue
                
            #compute implicit feedback
            N_u=np.where(~np.isnan(arr[:,u]))[0]
            N_u_norm_sqrt=arr[:,u][~np.isnan(arr[:,u])].shape[0]**0.5
            feedback=(y_j[N_u,:]/N_u_norm_sqrt).sum(axis=0)

            #compute error
            epsilon_ui=(r_ui-miu-b_u[u]-b_i[i]-q_i[i].T@(
                feedback+p_u[u]).reshape(-1,1)).item()
            error+=epsilon_ui

            #update
            b_u[u]+=alpha*(epsilon_ui-lambda_*b_u[u])
            b_i[i]+=alpha*(epsilon_ui-lambda_*b_i[i])
            p_u[u]+=alpha*(epsilon_ui*q_i[i]-lambda_*p_u[u])
            q_i[i]+=alpha*(epsilon_ui*(p_u[u]+feedback)-lambda_*q_i[i])
            y_j[N_u]+=alpha*(epsilon_ui*q_i[N_u]/N_u_norm_sqrt-lambda_*y_j[N_u])
               
    return error,b_u,b_i,p_u,q_i,y_j

In [12]:
#svdpp latent factor model
def svd_plus_plus(arr,miu_init=None,b_u_init=[],b_i_init=[],
             p_u_init=[],q_i_init=[],y_j_init=[],num_of_rank=40,
             alpha=0.005,lambda_=0.02,tau=0.0001,
             max_iter=20,diagnosis=True
             ):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
        
    #user latent factors
    if len(p_u_init)==0:
        p_u=np.zeros((arr.shape[1],num_of_rank))
        p_u.fill(0.1)
    else:
        p_u=p_u_init
    
    #item latent factors
    if len(q_i_init)==0:
        q_i=np.zeros((arr.shape[0],num_of_rank))
        q_i.fill(0.1)
    else:
        q_i=q_i_init
        
    #user implicit feedback
    if len(y_j_init)==0:
        y_j=np.zeros((arr.shape[0],num_of_rank))
        y_j.fill(0.1)
    else:
        y_j=y_j_init
    
    #gradient descent
    while not stop:
        
        error,b_u,b_i,p_u,q_i,y_j=svd_plus_plus_epoch(
            arr,miu,b_u,b_i,p_u,q_i,y_j,alpha,lambda_)

        counter+=1
        
        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged. Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error**2
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i,p_u,q_i,y_j

###### Run

In [13]:
#svdpp
b_u,b_i,p_u,q_i,y_j=svd_plus_plus(arr,
         num_of_rank=num_of_latent_factors,
         alpha=learning_rate,
         lambda_=lagrange_multiplier,tau=tolerance,
         max_iter=max_num_of_epoch)

44 iterations to reach convergence



In [14]:
#compute implicit feedback
feedback=[]
for u in range(arr.shape[1]):
    N_u=np.where(~np.isnan(arr[:,u]))[0]
    N_u_norm_sqrt=arr[:,u][~np.isnan(arr[:,u])].shape[0]**0.5
    feedback.append((y_j[N_u,:]/N_u_norm_sqrt).sum(axis=0))

In [15]:
#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),arr.shape[1],axis=1)+q_i@(p_u+np.mat(feedback)).T

In [16]:
#use mse as benchmark for comparison
mse_svdpp=np.square((
    output-arr)[~np.isnan(arr)]).sum()/len(arr[~np.isnan(arr)])
print('SVD++ Mean Squared Error:',round(mse_svdpp,3))

SVD++ Mean Squared Error: 0.701
